# Process lightning data from WWLN
This iPython notebook extracts lighning data from raw WWLN data files

Code by: Jasa Calogovic (Faculty of Geodesy, University of Zagreb) and Benjamin Laken (UCL)

Email: jcalogovic@geof.hr

In [1]:
# Load required packages
import numpy as np
import datetime as dt
from datetime import timedelta
import pandas as pd
import os
from bokeh.plotting import Figure, show, output_notebook, vplot
from bokeh.charts import Bar
from bokeh.io import gridplot
from bokeh.models import HoverTool, Span, Range1d, LinearAxis
from bokeh.models.sources import ColumnDataSource

from stormstats.storm import read_WWLN, get_map

output_notebook()

Loading BokehJS ...

## Define parameters

In [ ]:
#data_path='../data/WWLN/Jan2016/'
data_path = "WWLN"

#start_time=
#end_time=
time_step=60   # in minutes
west_long_limit=0
east_long_limit=30
north_lat_limit=60
south_lat_limit=30
residuals_limit_us=2
minimum_no_stations=5

# Load data

In [2]:
#test = read_WWLN(file="WWLN/test.loc")
test = read_WWLN(file="WWLN/A20160101.loc")
print("There are {0} rows in the dataframe.".format(len(test)))
test.head()

,datetime,lat,lon,err,#sta
0,2016-01-01 00:00:00.258574,-7.6253,-64.6813,13.1,6
1,2016-01-01 00:00:00.205713,-5.1888,23.3846,4.1,5
2,2016-01-01 00:00:00.297611,-7.4574,-64.7405,16.9,6
3,2016-01-01 00:00:00.996208,-6.9209,-60.5694,6.2,13
4,2016-01-01 00:00:00.996221,-6.8842,-60.5744,21.0,6


## Map data ## 

In [4]:
# mapping function doesnt seem to handel the large sizes well, so I am limiting the size for now
mx = get_map(strike_data = test)
mx

Warning, you have requested a large amount of data be mapped
I am limiting your request to the first 1,000 rows, as this
is currently only a preview feature.
